1. The Model
=================


In [48]:
import tensorflow as tf
from keras.layers import Dense, Flatten, Lambda, Activation, MaxPooling2D
from keras.layers.convolutional import Convolution2D
from keras.models import Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


In [50]:
tf.python.control_flow_ops = tf

In [ ]:
number of 

In [49]:
import errno
import json
import os

import cv2
%pylab inline

import pandas as pd
import scipy.misc
from scipy.ndimage import rotate
from scipy.stats import bernoulli
from scipy import signal

import glob
import math
import matplotlib.gridspec as gridspec
import random
from scipy import ndimage
import skimage

Populating the interactive namespace from numpy and matplotlib


C:\Users\qubit\Anaconda2\envs\py3\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
DRIVING_LOG_FILE = './data/driving_log.csv'
IMG_PATH = './data/'
STEERING_COEFFICIENT = 0.229

Image Preprocessing:
==============================================

## Helper Function For Cropping image.


In [3]:
def crop(image, top_percent, bottom_percent):
    """
    Function crops an image according to the given parameters.
    
    Args:
    - Image: source image
    - top_percent: The percentage of the original image will be croppped from the
                    top of the image.
    - bottom_percent: The percentage of the original image will be cropped from the 
                      bottom of the image
                      
    Return:
    - Cropped image
    
    """
    
    assert 0 <= top_percent < 0.5, 'top percent should be between 0.0 and 0.5'
    assert 0 <= bottom_percent < 0.5, 'top_percent should be between 0.0 and 0.5'
    
    top = int(np.ceil(image.shape[0] * top_percent))
    bottom = image.shape[0] - int(np.ceil(image.shape[0] * bottom_percent))
    
    return image[top:bottom, :]

## Helper function for Resizing image.

In [4]:
def resize(image, new_dim):
    """
    Resize a given image according to the new dimension
    Args:
    - image: source image
    - new_dim: A tuple which represents the resize dimension
    
    Return:
    - Resized image
    """
   
    return scipy.misc.imresize(image, new_dim)


## Helper function for Random flipping of image

In [5]:
def random_flip(image, steering_angle, flipping_prob=0.5):
    """
    Based on a random coin tossing, the image is flipped.
    If flipping is applied, the steering angle is negated.
    
    Args:
    - image: source image
    - steering_angle: Original steering angle
    
    Return:
    - Both flipped image and new steering angle
    """
    
    head = bernoulli.rvs(flipping_prob)
    
    if head:
        return np.flipr(image), -1 * steering_angle
    else:
        return image, steering_angle

In [51]:
def random_gamma(image):
    """
    Random gamma correction is used as an alternative method changing
    the brightness of training images.
    
    Args:
    - image: source image
    
    Return:
    - New image generated by applying gamma correction to the source image 
    
    """
    
    gamma = np.random.uniform(0.4, 1.5)
    inv_gamma = 1.0 / gamma
    table = np.array([(( i / 255.0) ** inv_gamma) * 255
                     for i in np.arange(0,256)]).astype("uint8")
    
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)


In [2]:
def random_shear(image, steering_angle, shear_range=200):